In [63]:
# Importação de bibliotecas

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

In [64]:
# Carregamento dos dados do arquivo CSV

df = pd.read_csv("../data/raw/titanic.csv")
display(df.head())

# Carregamento do dicionário de dados

df_dict = pd.read_csv("../data/external/dictionary.csv")
display(df_dict)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


,variavel,descrição,tipo,subtipo
0,survived,Indica se o passageiro sobreviveu (sim = 1 ou ...,qualitativa,nominal
1,pclass,"Classe do ticket do passageiro (1 = 1ª, 2 = 2ª...",qualitativa,ordinal
2,sex,Sexo do passageiro,qualitativa,nominal
3,age,Idade do passageiro,quantitativa,contínua
4,sibsp,Número de irmãos/cônjuges a bordo,quantitativa,discreta
5,parch,Número de pais/filhos a bordo,quantitativa,discreta
6,fare,Tarifa paga pelo passageiro,quantitativa,contínua
7,embarked,Porto de embarque (C = Cherbourg; Q = Queensto...,qualitativa,nominal
8,class,"Classe do ticket (Primeira, Segunda, Terceira)",qualitativa,ordinal
9,who,"Descrição da pessoa (homem, mulher, criança)",qualitativa,nominal


In [65]:
# Definição da variável alvo

target_variable = 'survived'

# Seleciona variáveis quantitativas, excluindo a variável alvo

quantitative_variables = (
    df_dict
    .query("tipo == 'quantitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Seleciona variáveis qualitativas, excluindo a variável alvo

qualitative_variables = (
    df_dict
    .query("tipo == 'qualitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [66]:
# Removendo a coluna da variável alvo do DataFrame original
X = df.drop(columns=[target_variable])

# Criar o DataFrame contendo apenas a coluna da variável alvo do DataFrame original
y = df[[target_variable]]

In [67]:
# tratamento de dados discrepantes
quantitative_preprocess = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # tratamento de dados ausentes
    ('normalization', StandardScaler()), # normalização
])

qualitative_preprocess = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados ausentes
    ('encoding', OneHotEncoder()), # transformação de variáveis
    # ('normalization', StandardScaler()), # normalização
])

preprocess = ColumnTransformer([
    ('quantitative', quantitative_preprocess, quantitative_variables),
    ('qualitative', qualitative_preprocess, qualitative_variables)
])

In [68]:
# Pré-processamento aos dados

X_preprocessed = preprocess.fit_transform(X)

print(X_preprocessed.shape)
print(X_preprocessed[:5])

(891, 34)
[[-0.56573646  0.43279337 -0.47367361 -0.50244517  0.          0.
   1.          0.          1.          0.          0.          1.
   0.          0.          1.          0.          1.          0.
   0.          1.          0.          0.          1.          0.
   0.          0.          0.          0.          0.          1.
   1.          0.          1.          0.        ]
 [ 0.66386103  0.43279337 -0.47367361  0.78684529  1.          0.
   0.          1.          0.          1.          0.          0.
   1.          0.          0.          0.          0.          1.
   1.          0.          0.          0.          1.          0.
   0.          0.          0.          1.          0.          0.
   0.          1.          1.          0.        ]
 [-0.25833709 -0.4745452  -0.47367361 -0.48885426  0.          0.
   1.          1.          0.          0.          0.          1.
   0.          0.          1.          0.          0.          1.
   1.          0.          0. 

In [69]:
# Definindo os nomes das colunas processadas após a aplicação do pipeline

quantitative_columns = quantitative_variables
qualitative_columns = preprocess.named_transformers_['qualitative']['encoding'].get_feature_names_out(qualitative_variables)

preprocessed_columns = list(quantitative_columns) + list(qualitative_columns)

In [70]:
# Convertendo os dados pré-processados em um DataFrame com os nomes das colunas

X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=preprocessed_columns)

display(X_preprocessed_df.head())

,age,sibsp,parch,fare,pclass_1,pclass_2,pclass_3,sex_female,sex_male,embarked_C,...,deck_E,deck_F,deck_G,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,alive_no,alive_yes,alone_False,alone_True
0,-0.565736,0.432793,-0.473674,-0.502445,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,0.663861,0.432793,-0.473674,0.786845,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-0.258337,-0.474545,-0.473674,-0.488854,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,0.433312,0.432793,-0.473674,0.420730,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,0.433312,-0.474545,-0.473674,-0.486337,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [71]:
# Verificação de variáveis com dados faltantes

missing_data_before = df.isnull().sum()
missing_data_before = missing_data_before[missing_data_before > 0]

print(missing_data_before)

age            177
embarked         2
deck           688
embark_town      2
dtype: int64


In [72]:
# Tratamento de dados faltantes

# Remoção a coluna 'deck' devido ao grande número de valores ausentes
df.drop(columns=['deck'], inplace=True)

# Imputação de valores ausentes na coluna 'age' com a mediana
df['age'].fillna(df['age'].median(), inplace=True)

# Imputar valores ausentes na coluna 'embarked' com a moda
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)

# Imputar valores ausentes na coluna 'embark_town' com a moda
df['embark_town'].fillna(df['embark_town'].mode()[0], inplace=True)

# Verificando se ainda existem dados faltantes
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [73]:
# Identificação de outliers usando o método do IQR (Interquartile Range)

def find_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data < lower_bound) | (data > upper_bound)]

# Dicionário para armazenar variáveis com outliers

outliers_dict = {}

# Verificar outliers em todas as variáveis numéricas

for column in df.select_dtypes(include=[np.number]).columns:
    outliers = find_outliers_iqr(df[column])
    if not outliers.empty:
        outliers_dict[column] = outliers

# Exibição resultados

if outliers_dict:
    print("Variáveis com outliers:")
    for column, outliers in outliers_dict.items():
        print(f"\nColuna '{column}' tem {len(outliers)} outliers:\n{outliers}\n")
else:
    print("")

Variáveis com outliers:

Coluna 'age' tem 66 outliers:
7       2.00
11     58.00
15     55.00
16      2.00
33     66.00
       ...  
827     1.00
829    62.00
831     0.83
851    74.00
879    56.00
Name: age, Length: 66, dtype: float64


Coluna 'sibsp' tem 46 outliers:
7      3
16     4
24     3
27     3
50     4
59     5
63     3
68     4
71     5
85     3
88     3
119    4
159    8
164    4
171    4
176    3
180    8
182    4
201    8
229    3
233    4
261    4
266    4
278    4
324    8
341    3
374    3
386    5
409    3
480    5
485    3
541    4
542    4
634    3
642    3
683    5
686    4
726    3
787    4
792    8
813    4
819    3
824    4
846    8
850    4
863    8
Name: sibsp, dtype: int64


Coluna 'parch' tem 213 outliers:
7      1
8      2
10     1
13     5
16     1
      ..
871    1
879    1
880    1
885    5
888    2
Name: parch, Length: 213, dtype: int64


Coluna 'fare' tem 116 outliers:
1       71.2833
27     263.0000
31     146.5208
34      82.1708
52      76.7292
   